In [41]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/News_mongo_redis/notebook


In [265]:
import sys
sys.path.append("")

from datetime import datetime
import requests
from dotenv import load_dotenv
load_dotenv()
import os

from src.redis_database import *
from src.crawl_and_summarize_news import *

# Crawl news

Ở đây ta crawl hết báo từ 2 trang ZingNews và CafeF (Chỉ crawl khoảng 100 bài). Một bài báo đầy đủ có các thành phần 
- 'title': "Bộ đôi iPhone mãi vẫn không bị 'khai tử' ở Việt Nam",
- 'link': 'https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html',
- 'authors': ['Xuân Sang'],
- 'language': 'vi',
- 'description': 'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.',
- 'publish_date': : datetime.datetime(2024, 2, 2, 18, 46, 52, tzinfo=tzoffset(None, 25200))

In [43]:
news_scraper = NewsScraper()


In [44]:
formatted_docs = news_scraper.get_all_news()

Crawling url: https://znews.vn/
Crawling url: https://cafef.vn/


In [45]:
print(formatted_docs[2][0].page_content)
print(formatted_docs[2][0].metadata)

Lằn ranh về sống chết là một thứ rất mong manh. Đừng nghĩ ngợi quá nhiều về cái chết, bởi đôi khi đó lại là khởi đầu cho một hành trình mới.

Đừng sợ hãi về cái chết, hãy tận hưởng cuộc sống với thái độ tích cực. Ảnh: NLĐ.

[...]

Khi trưởng thành, việc già đi nhắc chúng ta nhớ về sự mất mát; do đó, nó buộc chúng ta nhìn vào khoảng trống này, thứ đã bị mở ra vào thời thơ ấu. Giải phóng mình khỏi cái chết là mục tiêu của mọi tôn giáo: “Ôi cái chết, ngòi của ngươi ở đâu? Ôi nấm mồ, chiến thắng của ngươi ở đâu?”.

Thánh Paul đã hỏi như vậy trong bức thư gửi cho các tín hữu ở Corinth. Sau đó, ông đưa ra câu trả lời: “Ngòi của cái chết là tội lỗi”, ý chỉ sai lầm hoặc sự sa ngã. Nói cách khác, cái chết là kết quả của việc con người tách rời khỏi cái Thánh thần, cái không biết đến cái chết. Ngụ ý ở đây là sự bất tử mới là đời sống thực của chúng ta.

Thế con chim, con mèo hay con chó mà chết khi chúng ta còn nhỏ thì sao? Chúng diệt vong mà không hề có những niềm tin tôn giáo như vậy. Tuy nhiê

Cách lấy từng element của 1 bài báo

In [111]:
full_news = news_scraper.take_news_elements(news_url='https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html')

In [112]:
full_news[0].page_content

'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.\n\nKhông còn xuất hiện trên web Apple hay các thị trường lớn, iPhone 11, iPhone 12 vẫn được bán tại Việt Nam. Ảnh: PhoneArena.\n\nKhi ra mắt dòng iPhone 15 hồi tháng 9/2023, Apple âm thầm loại bỏ iPhone 12 khỏi danh sách thiết bị hãng còn sản xuất và kinh doanh. Mẫu iPhone 11 cũng bị “khai tử” trước đó một năm, cả hai không còn hiện diện trên cửa hàng trực tuyến của hãng. Tuy nhiên, hai chiếc máy nói trên vẫn đang được bán rộng rãi tại thị trường Việt Nam thông các kênh bán lẻ.\n\nTrong đó, chiếc iPhone 11 hiện có giá niêm yết khoảng 10 triệu đồng, không có nhiều chênh lệch giữa các đại lý. Phiên bản 64 GB của model nói trên là chiếc iPhone chính hãng giá rẻ nhất đang còn bán tại Việt Nam. Cao hơn khoảng 3 triệu đồng là chiếc iPhone 12 bộ nhớ mặc định. Cả hai mẫu máy này đều không có mặt trên trang web chính thức của Apple.\n\nMẫu máy 5 năm tuổi vẫn được sản xu

In [113]:
full_news[0].metadata

{'title': "Bộ đôi iPhone mãi vẫn không bị 'khai tử' ở Việt Nam",
 'link': 'https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html',
 'authors': ['Xuân Sang'],
 'language': 'vi',
 'description': 'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.',
 'publish_date': datetime.datetime(2024, 2, 2, 18, 46, 52, tzinfo=tzoffset(None, 25200))}

# News Summary

Tạo thử data Summary để lưu vào Redis. Giả sử đây là data lấy được từ MongoDB

In [114]:
input_dict = full_news[0].metadata
full_news = {
    'title': input_dict['title'],
    'link': input_dict['link'],
    'authors': ', '.join(input_dict['authors']),
    'publish_date': input_dict['publish_date'].strftime('%Y-%m-%d'),
    'page_content': full_news[0].page_content,
}

In [115]:
new_summarizer = NewsSummarizer()
sum_news = new_summarizer.summary_news_for_redis(full_news= full_news)

/root/code_Bao/News_mongo_redis/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [116]:
sum_news

{'title': "Bộ đôi iPhone mãi vẫn không bị 'khai tử' ở Việt Nam",
 'link': 'https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html',
 'authors': 'Xuân Sang',
 'publish_date': '2024-02-02',
 'page_content': 'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.\n\nKhông còn xuất hiện trên web Apple hay các thị trường lớn, iPhone 11, iPhone 12 vẫn được bán tại Việt Nam. Ảnh: PhoneArena.\n\nKhi ra mắt dòng iPhone 15 hồi tháng 9/2023, Apple âm thầm loại bỏ iPhone 12 khỏi danh sách thiết bị hãng còn sản xuất và kinh doanh. Mẫu iPhone 11 cũng bị “khai tử” trước đó một năm, cả hai không còn hiện diện trên cửa hàng trực tuyến của hãng. Tuy nhiên, hai chiếc máy nói trên vẫn đang được bán rộng rãi tại thị trường Việt Nam thông các kênh bán lẻ.\n\nTrong đó, chiếc iPhone 11 hiện có giá niêm yết khoảng 10 triệu đồng, không có nhiều chênh lệch giữa các đại lý. Phiên bản 64 GB của model nói trên là chi

# Save data in Redis

In [266]:
redis_db = RedisDatabase()
redis_db.flush_database()
redis_db.initilize_schema()

Connected: True
Database flushed at 2024-02-03 10:32:56.078371


In [267]:
sum_news

{'title': "'Đại gia' nào đang hiện diện trên thị trường bệnh viện tư nhân?",
 'link': 'https://znews.vn/dai-gia-nao-dang-hien-dien-tren-thi-truong-benh-vien-tu-nhan-post1453268.html',
 'authors': 'Thanh Thương',
 'publish_date': '2024-01-08',
 'page_content': 'Cuộc chiến tranh giành thị phần trong lĩnh vực chăm sóc sức khỏe đang "nóng" hơn khi ngày càng nhiều bệnh viện tư nhân được các doanh nghiệp trong và ngoài nước đầu tư mở rộng.\n\nHơn chục năm về trước, thị trường chăm sóc sức khỏe tại Việt Nam chủ yếu là sân chơi của các doanh nghiệp nội.\n\nNhưng đến nay, nhiều ông lớn ngoại đã và đang gia nhập vào cuộc đua giành vị thế trong lĩnh vực y tế bằng nhiều thương vụ mua bán và sáp nhập (M&A) đình đám.\n\nNhững bệnh viện 100% vốn ngoại tại Việt Nam\n\nBệnh viện Việt Pháp (Hà Nội) thành lập năm 1997 là bệnh viện 100% vốn nước ngoài đầu tiên ở Việt Nam. Từ năm 2000, công ty Eukaria (Pháp) tiếp quản bệnh viện sau khi mua lại cổ phần của Công ty Indochina Medical (IMC) của Australia. Sau 

In [268]:
redis_db.save_news(full_news=sum_news)

Giả sử ta lưu 6 bài báo top news vào redis

In [269]:
full_news_list = []
for i, document in enumerate(formatted_docs[:2]):
    print(document)
    input_dict = document[0].metadata
    print(input_dict)
    full_news = {
        'title': input_dict['title'],
        'link': input_dict['link'],
        'authors': ', '.join(input_dict['authors']),
        'publish_date': input_dict['publish_date'].strftime('%Y-%m-%d'),
        'page_content': document[0].page_content,
    }
    sum_news = new_summarizer.summary_news_for_redis(full_news= full_news)
    full_news_list.append(sum_news)
    
redis_db.save_news_in_batch(full_news_list=full_news_list)

[Document(page_content='Trong sách “Những khoảnh khắc sống”, TS Lê Kiên Thành (con trai cố Tổng bí thư Lê Duẩn) kể về gia đình mình và những câu chuyện, những nhân vật của một thời đã qua.\n\nSách Những khoảnh khắc sống. Ảnh: Y Nguyên.\n\nTiến sĩ Lê Kiên Thành (1955) từng học trường lái máy bay, kỹ sư không quân, nghiên cứu sinh tại Viện Nghiên cứu Hạt nhân Dupna, Liên Xô. Ông có 11 năm phục vụ trong quân đội, binh chủng Phòng không - Không quân, là một doanh nhân từ năm 1993.\n\nNhững bài viết trên báo chí, mạng xã hội của ông đặt ra nhiều vấn đề, được nhiều người theo dõi. Nhưng ở cuốn sách mới phát hành - Những khoảnh khắc sống - Lê Kiên Thành trở thành người kể chuyện về những con người và một thời đã qua. Sách gồm 17 bài viết , được chia thành hai phần “Truyện” và “Tự sự”.\n\nThông qua những bài viết trong phần “Truyện”, đặc biệt là hai bài viết Làng ven sông, Gia đình má Chín Nhơn… tác giả kể câu chuyện điển hình về những năm tháng chiến tranh của dân tộc. Một truyện về làng quê 

/root/code_Bao/News_mongo_redis/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[Document(page_content='Cuộc chiến tranh giành thị phần trong lĩnh vực chăm sóc sức khỏe đang "nóng" hơn khi ngày càng nhiều bệnh viện tư nhân được các doanh nghiệp trong và ngoài nước đầu tư mở rộng.\n\nHơn chục năm về trước, thị trường chăm sóc sức khỏe tại Việt Nam chủ yếu là sân chơi của các doanh nghiệp nội.\n\nNhưng đến nay, nhiều ông lớn ngoại đã và đang gia nhập vào cuộc đua giành vị thế trong lĩnh vực y tế bằng nhiều thương vụ mua bán và sáp nhập (M&A) đình đám.\n\nNhững bệnh viện 100% vốn ngoại tại Việt Nam\n\nBệnh viện Việt Pháp (Hà Nội) thành lập năm 1997 là bệnh viện 100% vốn nước ngoài đầu tiên ở Việt Nam. Từ năm 2000, công ty Eukaria (Pháp) tiếp quản bệnh viện sau khi mua lại cổ phần của Công ty Indochina Medical (IMC) của Australia. Sau hơn 25 năm hoạt động, đến nay bệnh viện đã nâng công suất lên 170 giường bệnh.\n\nCũng sở hữu 100% vốn nước ngoài, bệnh viện FV (TP.HCM) do bác sĩ Jean-Marcel Guillon và nhóm bác sĩ người Pháp thành lập năm 2003. FV có số vốn điều lệ ban

/root/code_Bao/News_mongo_redis/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


## Query check

In [270]:
full_news_list

[{'title': 'Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành',
  'link': 'https://znews.vn/gia-dinh-co-tong-bi-thu-le-duan-qua-hoi-uc-cua-ts-le-kien-thanh-post1450645.html',
  'authors': 'Y Nguyên',
  'publish_date': '2024-01-31',
  'page_content': 'Trong sách “Những khoảnh khắc sống”, TS Lê Kiên Thành (con trai cố Tổng bí thư Lê Duẩn) kể về gia đình mình và những câu chuyện, những nhân vật của một thời đã qua.\n\nSách Những khoảnh khắc sống. Ảnh: Y Nguyên.\n\nTiến sĩ Lê Kiên Thành (1955) từng học trường lái máy bay, kỹ sư không quân, nghiên cứu sinh tại Viện Nghiên cứu Hạt nhân Dupna, Liên Xô. Ông có 11 năm phục vụ trong quân đội, binh chủng Phòng không - Không quân, là một doanh nhân từ năm 1993.\n\nNhững bài viết trên báo chí, mạng xã hội của ông đặt ra nhiều vấn đề, được nhiều người theo dõi. Nhưng ở cuốn sách mới phát hành - Những khoảnh khắc sống - Lê Kiên Thành trở thành người kể chuyện về những con người và một thời đã qua. Sách gồm 17 bài viết , được chia thành 

In [271]:
redis_db.save_news_in_batch(full_news_list=full_news_list)

In [272]:
redis_db.get_news_with_title(text='tổng bí thư')

Result{1 total, docs: [Document {'id': 'title:Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành', 'payload': None, 'json': '{"title":"Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành","link":"https://znews.vn/gia-dinh-co-tong-bi-thu-le-duan-qua-hoi-uc-cua-ts-le-kien-thanh-post1450645.html","authors":"Y Nguyên","publish_date":"2024-01-31","page_content":"Trong sách “Những khoảnh khắc sống”, TS Lê Kiên Thành (con trai cố Tổng bí thư Lê Duẩn) kể về gia đình mình và những câu chuyện, những nhân vật của một thời đã qua.\\n\\nSách Những khoảnh khắc sống. Ảnh: Y Nguyên.\\n\\nTiến sĩ Lê Kiên Thành (1955) từng học trường lái máy bay, kỹ sư không quân, nghiên cứu sinh tại Viện Nghiên cứu Hạt nhân Dupna, Liên Xô. Ông có 11 năm phục vụ trong quân đội, binh chủng Phòng không - Không quân, là một doanh nhân từ năm 1993.\\n\\nNhững bài viết trên báo chí, mạng xã hội của ông đặt ra nhiều vấn đề, được nhiều người theo dõi. Nhưng ở cuốn sách mới phát hành - Những khoảnh khắc

In [273]:
redis_db.get_news_with_page_content_match(text='tổng bí thư')


Result{1 total, docs: [Document {'id': 'title:Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành', 'payload': None, 'json': '{"title":"Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành","link":"https://znews.vn/gia-dinh-co-tong-bi-thu-le-duan-qua-hoi-uc-cua-ts-le-kien-thanh-post1450645.html","authors":"Y Nguyên","publish_date":"2024-01-31","page_content":"Trong sách “Những khoảnh khắc sống”, TS Lê Kiên Thành (con trai cố Tổng bí thư Lê Duẩn) kể về gia đình mình và những câu chuyện, những nhân vật của một thời đã qua.\\n\\nSách Những khoảnh khắc sống. Ảnh: Y Nguyên.\\n\\nTiến sĩ Lê Kiên Thành (1955) từng học trường lái máy bay, kỹ sư không quân, nghiên cứu sinh tại Viện Nghiên cứu Hạt nhân Dupna, Liên Xô. Ông có 11 năm phục vụ trong quân đội, binh chủng Phòng không - Không quân, là một doanh nhân từ năm 1993.\\n\\nNhững bài viết trên báo chí, mạng xã hội của ông đặt ra nhiều vấn đề, được nhiều người theo dõi. Nhưng ở cuốn sách mới phát hành - Những khoảnh khắc

In [274]:
res = redis_db.rs.search(Query(f"@page_content: Tổng bí thư"))
res

Result{1 total, docs: [Document {'id': 'title:Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành', 'payload': None, 'json': '{"title":"Gia đình cố Tổng bí thư Lê Duẩn qua hồi ức của TS Lê Kiên Thành","link":"https://znews.vn/gia-dinh-co-tong-bi-thu-le-duan-qua-hoi-uc-cua-ts-le-kien-thanh-post1450645.html","authors":"Y Nguyên","publish_date":"2024-01-31","page_content":"Trong sách “Những khoảnh khắc sống”, TS Lê Kiên Thành (con trai cố Tổng bí thư Lê Duẩn) kể về gia đình mình và những câu chuyện, những nhân vật của một thời đã qua.\\n\\nSách Những khoảnh khắc sống. Ảnh: Y Nguyên.\\n\\nTiến sĩ Lê Kiên Thành (1955) từng học trường lái máy bay, kỹ sư không quân, nghiên cứu sinh tại Viện Nghiên cứu Hạt nhân Dupna, Liên Xô. Ông có 11 năm phục vụ trong quân đội, binh chủng Phòng không - Không quân, là một doanh nhân từ năm 1993.\\n\\nNhững bài viết trên báo chí, mạng xã hội của ông đặt ra nhiều vấn đề, được nhiều người theo dõi. Nhưng ở cuốn sách mới phát hành - Những khoảnh khắc